In [38]:
from utils import extract_scaling_metadata
import os
from tqdm import tqdm
from pathlib import Path
import czifile
import napari
import numpy as np
import skimage
from skimage import measure, exposure
from skimage.measure import regionprops_table
import pandas as pd
import pyclesperanto_prototype as cle
import plotly.express as px
from cellpose import models, utils

cle.select_device("RTX")

<NVIDIA GeForce RTX 4090 on Platform: NVIDIA CUDA (2 refs)>

In [39]:
directory_path = Path("./raw_data/")
images = []

# Iterate through the lsm files in the directory
for file_path in directory_path.glob("*.czi"):
    # Remove Control and Isotype stainings from the analysis
    if "Control" and "Isotype" not in file_path.stem:
        images.append(str(file_path))
    
len(images)

117

In [40]:
image = images[1]

# Read path storing raw image and extract filename
file_path = Path(image)
filename = file_path.stem

# Read the image file and remove singleton dimensions
img = czifile.imread(image)
img = img.squeeze()

# Extract experiment, mouse, treatment and replica ids
experiment_id = filename.split(" ")[0]
mouse_id = filename.split(" ")[1]
treatment_id = filename.split(" ")[2]
replica_id = filename.split(" ")[-1]

# Image size reduction to improve processing times (slicing, not lossless compression)
slicing_factor = None # Use 2 or 4 for compression (None for lossless)

# Extract the stack containing the UEA-1 (0), YAP (1), nuclei (2) and BCAT (3) channels.
uea1_stack = img[0, :, ::slicing_factor, ::slicing_factor]
yap_stack = img[1, :, ::slicing_factor, ::slicing_factor]
nuclei_stack = img[2, :, ::slicing_factor, ::slicing_factor]
bcat_stack = img[3, :, ::slicing_factor, ::slicing_factor]

In [41]:
# Extract x,y,z scaling from .czi file metadata in order to make data isotropic
scaling_x_um, scaling_y_um, scaling_z_um = extract_scaling_metadata(file_path)

# Resample nuclei stack and remove noise prior to Cellpose prediction
resampled_nuclei = cle.scale(nuclei_stack, factor_x=scaling_x_um, factor_y=scaling_y_um, factor_z=scaling_z_um, auto_size=True)
background_subtracted = cle.top_hat_box(resampled_nuclei, radius_x=5, radius_y=5, radius_z=5)

# Pull OCL arrays from GPU memory and transform into numpy arrays to be fed into Cellpose
processed_nuclei = cle.pull(background_subtracted)
del background_subtracted

# Apply Contrast Stretching to improve Cellpose detection of overly bright nuclei
p2, p98 = np.percentile(processed_nuclei, (2, 98)) 
nuclei_rescaled = exposure.rescale_intensity(processed_nuclei, in_range=(p2, p98))

# Initialize the Cellpose model
model = models.Cellpose(gpu=True, model_type='nuclei')

# Run Cellpose
masks, flows, styles, diams = model.eval(nuclei_rescaled, diameter=10, channels=[0, 0], do_3D=True)


In [42]:
# Filter out merged nuclei labels and artifacts based on size distribution

# Create a destination array with the same shape as the nuclei labels (masks array)
destination = cle.create_like(masks)

# Remove labels smaller than 90 pixels (artifacts)
filtered_nuclei_gpu = cle.exclude_small_labels(masks, destination, 90)

# Remove labels larger than 500 pixels (merged nuclei)
filtered_nuclei_gpu = cle.exclude_large_labels(filtered_nuclei_gpu, destination, 500)

# Pull filtered_nuclei from GPU
filtered_nuclei = cle.pull(filtered_nuclei_gpu).astype(np.uint16)

# Dilate labels
dilated_nuclei_gpu = cle.dilate_labels(filtered_nuclei, radius=2)
dilated_nuclei = cle.pull(dilated_nuclei_gpu)


In [43]:
# Simulate cytoplasm labels by building a sphere surrounding the nuclei labels

# Create a copy of dilated_nuclei to modify
cytoplasm = dilated_nuclei.copy()

# Get unique labels (excluding 0 which is background)
unique_labels = np.unique(filtered_nuclei)
unique_labels = unique_labels[unique_labels != 0]

# Iterate over each label and remove the corresponding pixels from dilated_nuclei
for label in unique_labels:
    # Create a mask for the current label in filtered_nuclei
    mask = (filtered_nuclei == label)
    
    # Set corresponding pixels in resulting_nuclei to zero
    cytoplasm[mask] = 0

In [44]:
# Resample YAP and B-catenin stacks for visualization
resampled_yap_gpu = cle.scale(yap_stack, factor_x=scaling_x_um, factor_y=scaling_y_um, factor_z=scaling_z_um, auto_size=True)
resampled_bcat_gpu = cle.scale(bcat_stack, factor_x=scaling_x_um, factor_y=scaling_y_um, factor_z=scaling_z_um, auto_size=True)

# Pull YAP OCL array from GPU to feed into regionprops as intensity_image afterwards
resampled_yap = cle.pull(resampled_yap_gpu)

In [45]:
# Initialize Napari viewer to display segmentation results
viewer = napari.Viewer(ndisplay=2)

viewer.add_image(processed_nuclei)
viewer.add_image(resampled_yap_gpu)
viewer.add_image(resampled_bcat_gpu)
viewer.add_labels(masks)
viewer.add_labels(filtered_nuclei)
viewer.add_labels(cytoplasm)


<Labels layer 'cytoplasm' at 0x1a531bd7fd0>

In [46]:
#Extract regionprops from filtered nuclei and generated-cytoplasm labels
props_nuclei = regionprops_table(label_image=filtered_nuclei, intensity_image=resampled_yap, properties=["label", "intensity_mean", "area"])
props_cytoplasm = regionprops_table(label_image=cytoplasm, intensity_image=resampled_yap, properties=["label", "intensity_mean", "area"])


# Transform regionprops_table into a Dataframe to process it using Pandas
df_nuclei = pd.DataFrame(props_nuclei)
df_cytoplasm = pd.DataFrame(props_cytoplasm)

# Renaming columns
df_nuclei.rename(columns={'intensity_mean': 'intensity_mean_nuclei', 'area': 'area_nuclei'}, inplace=True)
df_cytoplasm.rename(columns={'intensity_mean': 'intensity_mean_cyto', 'area': 'area_cyto'}, inplace=True)

# Merge dataframes on label
merged_df = pd.merge(df_nuclei, df_cytoplasm, on='label')

# Calculate nuclei/cytoplasm ratio of mean intensity of YAP signal
merged_df['nuclei_cyto_ratio'] = merged_df['intensity_mean_nuclei'] / merged_df['intensity_mean_cyto']

In [47]:
# Add extracted ID info to the dataframe
merged_df.insert(0, 'experiment_id', experiment_id)
merged_df.insert(1, 'mouse_id', mouse_id)
merged_df.insert(2, 'treatment_id', treatment_id)
merged_df.insert(3, 'replica_id', replica_id)

In [48]:
merged_df.head()

,experiment_id,mouse_id,treatment_id,replica_id,label,intensity_mean_nuclei,area_nuclei,intensity_mean_cyto,area_cyto,nuclei_cyto_ratio
0,RY20240607,4541,BCM,2,1,12.798761,323.0,12.575000,520.0,1.017794
1,RY20240607,4541,BCM,2,2,21.879896,383.0,27.395294,425.0,0.798674
2,RY20240607,4541,BCM,2,3,19.874233,326.0,22.310427,422.0,0.890805
3,RY20240607,4541,BCM,2,4,24.190083,121.0,19.887180,195.0,1.216366
4,RY20240607,4541,BCM,2,5,19.056604,265.0,23.157429,451.0,0.822915


In [49]:
# Explore particular label results (i.e. 541)

label = 541

merged_df[merged_df['label']==label]

,experiment_id,mouse_id,treatment_id,replica_id,label,intensity_mean_nuclei,area_nuclei,intensity_mean_cyto,area_cyto,nuclei_cyto_ratio
540,RY20240607,4541,BCM,2,541,1.511628,473.0,1.193665,1105.0,1.266375


In [50]:
# Explore nuclei predicted labels by Cellpose to make an informed decision on the size-based exclusion of the predicted labels
props = regionprops_table(label_image=masks, intensity_image=processed_nuclei, properties=["label", "intensity_mean", "equivalent_diameter_area", "area"])

df = pd.DataFrame(props)

df[df['label']==153]

# Create histogram
fig = px.histogram(df, x="area", nbins=300, labels={"area": "Area"}, title="Histogram of Areas of Cellpose predicted labels")

# Show plot
fig.show()